# Import Libraries

In [1]:
!pip install faiss-cpu -q
!pip install git+https://github.com/cvg/LightGlue.git -q
!pip install pycolmap -q

!mkdir -p /root/.cache/torch/hub/checkpoints
# !cp -r /kaggle/working/.local/lib/python*/site-packages/* offline_packages/

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 61.1 MB/s eta 0:00:00:00:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 92.8 MB/s eta 0:00:00:00:010:01


In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from collections import defaultdict
from tqdm import tqdm
from itertools import combinations
import h5py
import dataclasses

import cv2
from PIL import Image

import torch
from transformers import AutoImageProcessor, AutoModel
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms
from typing import Optional, Tuple
import torch.nn.functional as F

import faiss
import networkx as nx
from community import community_louvain

from lightglue import LightGlue, SuperPoint, ALIKED
from lightglue.utils import rbd
import pycolmap
from torchvision import transforms as T

import sys, warnings
sys.path.append("/kaggle/input/imc25-utils")

from database import *
from h5_to_db import *
import metric
import shutil

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Callable, List, Optional, Tuple, Union
from pathlib import Path

2025-06-11 15:45:58.175856: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749656758.393803      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749656758.463267      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/lightglue/lightglue.py:24: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


# STEP 1

In [3]:
# to extract  global feature using DINO Vision Transformers (ViTs)
all_features = []
image_paths = []

def global_feature_extractor(
    device, 
    Dataloader,
    verbose = True,
    pooling = 'mac',
    model_path = '/kaggle/input/dinov2/pytorch/large/1'):
    
    assert pooling in ["mac", "cls"], "Pooling must be 'mac' or 'cls'"
    loop = tqdm(Dataloader, desc="Extracting DINO features", disable=not verbose)
    
    processor = AutoImageProcessor.from_pretrained(model_path,use_fast=True)
    dino_model = AutoModel.from_pretrained(model_path)
    dino_model = dino_model.eval().to(device)
    
    with torch.inference_mode():
        feature_list = []
        for images, metadata in loop:
            inputs = processor(images=images, return_tensors="pt",do_rescale=False).to(device)
            outputs = dino_model(**inputs)
            if pooling == 'mac':
                # last_hidden_state shape: (batch_size, sequence_length, hidden_dim)
                vec = F.normalize(outputs.last_hidden_state[:,1:].max(dim=1)[0], dim=1, p=2) #all
            else:
                vec = F.normalize(outputs.last_hidden_state[:,0], dim=1, p=2) # batch, hidden_dim
            feature_list.append(vec.detach().cpu())
            image_paths.extend(metadata['image_path'])
            
        all_feature_list = torch.cat(feature_list, dim=0)
        
    return all_feature_list.numpy(), image_paths

In [4]:
# to build FAISS knn graph from global features
def build_knn_graph(features, data_dir, img_paths, k=10):
    """
    Build a kNN graph using cosine similarity and per-scene threshold filtering.
    
    features: np.ndarray of global descriptors
    img_paths: List of paths aligned with features
    scene_thresholds: Dict[scene_name] -> float (e.g., lowest accepted threshold)
    k: number of neighbors

    Returns:
        List of (i, j, sim) edges that pass scene threshold check
    """
    def load_threshold(dir):
        """
        return threshold value per scene in a Dict
        """
        threshold = os.path.join(data_dir, "train_thresholds.csv")
        thresh_df = pd.read_csv(threshold)
        tdict = dict()
        for _, row in thresh_df.iterrows():
            tdict[row["scene"]] = float(row['thresholds'].split(';')[-1])
            
        return tdict
    
    
    features = features.astype(np.float32)
    features /= np.linalg.norm(features, axis=1, keepdims=True)

    index = faiss.IndexFlatIP(features.shape[1])  # inner product = cosine sim if normalized
    index.add(features)
    distances, indices = index.search(features, k + 1)  # +1 because self-match at index 0

    paths = [os.path.dirname(p).split("/")[-1] for p in img_paths]
    og_scene_threshold = load_threshold(data_dir)


    edges = []
    num_points = features.shape[0]
    for i in range(num_points):
        for j in range(1, k + 1):  # skip self-match
            neighbor = indices[i][j]
            sim = distances[i][j]

            scene_1 = paths[i]
            scene_2 = paths[neighbor]

            if scene_1 == scene_2:
                threshold = og_scene_threshold.get(scene_1, 0.1)
                if sim >= threshold:
                    edges.append((i, neighbor, sim))
                else:
                    continue

    return edges

In [5]:
def build_graph_from_edges(edges):
    """
    Builds a weighted undirected NetworkX graph from edge list (i, j, weight)
    """
    G = nx.Graph()
    for i, j, sim in edges:
        G.add_edge(i, j, weight=sim)
    return G

def louvian_cluster(graph):
    return community_louvain.best_partition(graph,weight = 'weight')

# STEP 2

In [6]:
# superpoint + lightglue -> match_pair [reformat]

In [7]:
# img_load -> from lightglue util.py

def read_image(path, grayscale: bool = False) -> np.ndarray:
    """Read an image from path as RGB or grayscale"""
    if not Path(path).exists():
        raise FileNotFoundError(f"No image at path {path}.")
    mode = cv2.IMREAD_GRAYSCALE if grayscale else cv2.IMREAD_COLOR
    image = cv2.imread(str(path), mode)
    if image is None:
        raise IOError(f"Could not read image at {path}.")
    if not grayscale:
        image = image[..., ::-1]
    return image

def resize_image(
    image: np.ndarray,
    size: Union[List[int], int],
    fn: str = "max",
    interp: Optional[str] = "area",
) -> np.ndarray:
    """Resize an image to a fixed size, or according to max or min edge."""
    h, w = image.shape[:2]

    fn = {"max": max, "min": min}[fn]
    if isinstance(size, int):
        scale = size / fn(h, w)
        h_new, w_new = int(round(h * scale)), int(round(w * scale))
        scale = (w_new / w, h_new / h)
    elif isinstance(size, (tuple, list)):
        h_new, w_new = size
        scale = (w_new / w, h_new / h)
    else:
        raise ValueError(f"Incorrect new size: {size}")
    mode = {
        "linear": cv2.INTER_LINEAR,
        "cubic": cv2.INTER_CUBIC,
        "nearest": cv2.INTER_NEAREST,
        "area": cv2.INTER_AREA,
    }[interp]
    return cv2.resize(image, (w_new, h_new), interpolation=mode), scale


def numpy_image_to_torch(image: np.ndarray) -> torch.Tensor:
    """Normalize the image tensor and reorder the dimensions."""
    if image.ndim == 3:
        image = image.transpose((2, 0, 1))  # HxWxC to CxHxW
    elif image.ndim == 2:
        image = image[None]  # add channel axis
    else:
        raise ValueError(f"Not an image: {image.shape}")
    return torch.tensor(image / 255.0, dtype=torch.float)

def load_image(path , resize: int = None, **kwargs) -> torch.Tensor:
    image = read_image(path)
    if resize is not None:
        image, _ = resize_image(image, resize, **kwargs)
    return numpy_image_to_torch(image)

In [8]:
def match_pair_lightglue_superpoint(
    feature_dir,
    extractor,
    matcher,
    image_fnames,
    image_pairs,
    device,
    min_matches=20
):
    os.makedirs(feature_dir, exist_ok=True)
    
    with h5py.File(f'{feature_dir}/keypoints.h5', 'a') as f_kp, \
         h5py.File(f'{feature_dir}/descriptors.h5', 'a') as f_desc, \
         h5py.File(f'{feature_dir}/matches.h5', 'a') as f_match:

        for path1, path2 in tqdm(image_pairs, desc="Extracting & Matching"):
            key1 = os.path.basename(path1)
            key2 = os.path.basename(path2)

            # Always store under key1/key2 sorted order
            keyA, keyB = sorted([key1, key2])
            pathA, pathB = (path1, path2) if key1 == keyA else (path2, path1)

            if keyA in f_match and keyB in f_match[keyA]:
                continue

            try:
                imgA = load_image(pathA)
                imgB = load_image(pathB)

                featsA = extractor.extract(imgA.to(device))
                featsB = extractor.extract(imgB.to(device))

                matchesAB = matcher({"image0": featsA, "image1": featsB})
                featsA, featsB, matchesAB = [
                    rbd(x) for x in [featsA, featsB, matchesAB]]# remove batch_dimension

                matches = matchesAB["matches"]

                if matches.shape[0] < min_matches:
                    continue

                # Save matches
                f_match.require_group(keyA).create_dataset(keyB, data=matches.cpu().numpy().astype(np.int32))

                # Save keypoints/descriptors only once
                if keyA not in f_kp:
                    f_kp.create_dataset(keyA, data=featsA["keypoints"].cpu().numpy())
                    f_desc.create_dataset(keyA, data=featsA["descriptors"].cpu().numpy())

                if keyB not in f_kp:
                    f_kp.create_dataset(keyB, data=featsB["keypoints"].cpu().numpy())
                    f_desc.create_dataset(keyB, data=featsB["descriptors"].cpu().numpy())

            except Exception as e:
                print(f"[ERROR] Matching failed for {key1} ↔ {key2}: {e}")
                continue


# STEP 3

In [9]:
# COLMAP & pycolmap

In [10]:
def run_colmap_reconstruction(
    cluster_imgs,
    feature_dir,
    cluster_idx,
    output_root
):
    """
    Performs COLMAP-based SfM for a given cluster.
    Steps:
      - Creates a working directory
      - Copies required feature files and renames inliers.h5 → matches.h5
      - Imports images, keypoints, matches into COLMAP
      - Runs incremental mapping
      - Returns pose dictionary per image (rotation, translation)
    """

    # === Paths ===
    os.makedirs(output_root, exist_ok=True)

    db_path = os.path.join(output_root, "colmap.db")
    print(db_path)
    if os.path.exists(db_path):
        os.remove(db_path)

    base_cluster_idic = {}
    for i_path in cluster_imgs:
        bkey = os.path.basename(i_path)
        base_cluster_idic[bkey] = i_path
    
    # print(base_cluster_idic)

    def add_keypoints_flat(db, feature_dir, img_root, img_ext, camera_model, single_camera):
        f_kp = h5py.File(os.path.join(feature_dir, "keypoints.h5"), "r")
        fname_to_id = {}
        camera_id = None
        count = 0
        
        for img_name in tqdm(list(f_kp.keys()), desc="Add_Keypoints"):
            if img_name in base_cluster_idic.keys():
                full_img_path = base_cluster_idic[img_name]
                scene_img_path = os.path.relpath(full_img_path, img_root)
                # count+=1
                # if count<5:
                #     print(scene_img_path)
                
                if not os.path.isfile(full_img_path):
                    continue
            
                keypoints = f_kp[img_name][()]
                
                if len(keypoints) == 0:
                    print(f"for image {img_name} 0 keypoints")
                    continue
                
                if camera_id is None or not single_camera:
                    camera_id = create_camera(db, full_img_path, camera_model)
                    
                image_id = db.add_image(scene_img_path, camera_id)
                db.add_keypoints(image_id, keypoints)
                fname_to_id[img_name] = image_id
    
        return fname_to_id


    def add_matches_flat(db, feature_dir, fname_to_id):
        # print(fname_to_id)
        f_match = h5py.File(os.path.join(feature_dir, "matches.h5"), "r")
        added = set()
        n_total  = 0
        valid_pairs = 0
    
        # Estimate total number of match entries for tqdm
        n_keys = len(f_match.keys())
        n_total = (n_keys * (n_keys - 1)) // 2

        # count = 0
        with tqdm(total=n_total, desc="Importing Matches") as pbar:         
            for key1 in f_match.keys():
                key_group = f_match[key1] 
                for key2 in key_group.keys():
                    
                    if key1 not in fname_to_id and key2 not in fname_to_id:
                        continue

                    id_1 = fname_to_id[key1]
                    id_2 = fname_to_id[key2]
                    pair_id = image_ids_to_pair_id(id_1, id_2)
                    if pair_id in added:
                        warnings.warn(f'Pair {pair_id} ({id_1}, {id_2}) already added!')
                        pbar.update(1)
                        continue
                        
                    matches = key_group[key2][()] 
                    
                    # count += 1
                    # if count<= 10:
                    #     print(key1, key2)
                    #     print(f"Adding match: {key1} ↔ {key2}, shape: {matches.shape}, dtype: {matches.dtype}") # sanity check
                    
                    matches = np.unique(matches.astype(np.uint32), axis=0)
                    if matches.shape[0] == 0 or matches.ndim != 2 or matches.shape[1]!=2:
                        pbar.update(1)
                        continue
                    db.add_matches(id_1, id_2, matches.astype(np.uint32))
                    db.add_two_view_geometry(id_1, id_2, matches)
                    added.add(pair_id)
                    pbar.update(1)
                    valid_pairs += 1
        
        print(f"[DEBUG] Total valid pairs added to DB: {valid_pairs}")
                        

    
    def import_into_colmap(img_list, feature_dir, database_path):
        
        db = COLMAPDatabase.connect(database_path)
        db.create_tables()
        
        fname_to_id = add_keypoints_flat(
            db,
            feature_dir,
            img_root="/kaggle/input/image-matching-challenge-2025/train",
            img_ext=".png", camera_model='pinhole', single_camera=False)
    
        add_matches_flat(
            db,
            feature_dir,
            fname_to_id
        )
    
        db.commit()
        return
    
    import_into_colmap(cluster_imgs, feature_dir, db_path)
    image_dir = "/kaggle/input/image-matching-challenge-2025/train"

    # RANSAC - pycolmap ver
    # pycolmap.match_exhaustive(db_path)
        
    # # === Step 3: Incremental SfM ===
    
    maps = dict()
    mapper_options = pycolmap.IncrementalPipelineOptions()
    mapper_options.min_model_size = 8 #3
    mapper_options.max_num_models = 25 
    
    maps = pycolmap.incremental_mapping(
        database_path=db_path,
        image_path=image_dir,
        output_path=output_root,
        options=mapper_options
    )


    return maps

# DATALOADER

In [11]:
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [12]:
# dataloader for the whole pipeline
class ImageMatching():
    def __init__(
        self,
        csv_path: str,
        image_dir: str,
        to_train: bool = True,
        datasets_filter: Optional[list] = None,
        image_transform: Optional[transforms.Compose] = None
        ):
        self.data = []
        self.is_train = to_train
        self.data_dir = image_dir
        self.image_transform = image_transform 
        df = pd.read_csv(csv_path)

        for _, row in df.iterrows():
            if row['dataset'] in datasets_filter:
                entry = {
                    'dataset': row['dataset'],
                    'scene': row['scene'],
                    'filename': row['image'],
                    'image_id': row.get('image_id', None),  
                    'rotation': row.get('rotation_matrix', None) if to_train else None,
                    'translation': row.get('translation_vector', None) if to_train else None,
                }
                self.data.append(entry)
            

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx:int):
        entry = self.data[idx]
        base_dir = 'train' if self.is_train else 'test'
        image_path = os.path.join(self.data_dir, base_dir, entry['dataset'], entry['filename'])
        img = Image.open(image_path).convert("RGB")
        img = self.image_transform(img)

        metadata = {
            'image_path': image_path,
            'dataset': entry['dataset'],
            'scene': entry['scene'],
            'filename': entry['filename'],
        }

        if not self.is_train:
            metadata['image_id'] = entry['image_id']
                
        else:
            metadata['rotation_matrix'] = entry['rotation']
            metadata['translation_vector'] = entry['translation']
        
        return img, metadata
        

In [13]:
@dataclasses.dataclass
class Predictions:
    image_id: Optional[str]  # test mode only
    dataset: str
    filename: str
    cluster_index: Optional[int] = None
    rotation: Optional[np.ndarray] = None
    translation: Optional[np.ndarray] = None

image_dir = "/kaggle/input/image-matching-challenge-2025/"
res_dir = "/kaggle/working/result"
to_train = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_csv = os.path.join(image_dir,'train_labels.csv')
feature_dir = os.path.join(res_dir, 'featureout')
colmap_output_dir = os.path.join(res_dir, "output/colmap")
# print(device)

In [14]:
# for testing

#to pass in the images
# dataset = ImageMatching(
#             train_csv,
#             image_dir,
#             to_train,
#             datasets_filter=['imc2023_haiper','stairs''imc2024_dioscuri_baalshamin','imc2023_theather_imc2024_church'], #'imc2023_haiper','stairs''imc2024_dioscuri_baalshamin','imc2023_theather_imc2024_church'
#             image_transform= image_transform)

# train_Dataloader = DataLoader(dataset, batch_size=16, shuffle=False)

# for img_tensor, meta in train_Dataloader:
#     print(meta['filename'])
#     print(meta['image_path'])
#     print(img_tensor.shape)    
#     break

# Flow

In [15]:
def generate_image_pairs(clusters):
    """
    Returns a set of unique image pairs using canonical (sorted) tuple to prevent (i,j) vs (j,i) duplication.
    """
    unique_pairs = set()
    for cluster_id, img_list in clusters.items():
        if len(img_list) < 2:
            continue
        for img1, img2 in combinations(img_list, 2):
            pair = tuple(sorted((img1, img2)))  # prevent flipped duplicates
            unique_pairs.add(pair)
    return list(unique_pairs)

In [16]:
dataset = ImageMatching(
            train_csv,
            image_dir,
            to_train,
            datasets_filter=['imc2023_haiper','stairs''imc2024_dioscuri_baalshamin','imc2023_theather_imc2024_church'], #'imc2023_haiper','stairs''imc2024_dioscuri_baalshamin','imc2023_theather_imc2024_church'
            image_transform= image_transform)

train_Dataloader = DataLoader(dataset, batch_size=16, shuffle=False)

#step1
feature, img_paths = global_feature_extractor(device, train_Dataloader)
edges = build_knn_graph(feature, image_dir, img_paths, k=15)
Graph = build_graph_from_edges(edges)
cluster_map = louvian_cluster(Graph)

clusters = defaultdict(list)
for idx, cluster_id in cluster_map.items():
    clusters[cluster_id].append(img_paths[idx])

# step2

image_pairs = generate_image_pairs(clusters)

extractor = SuperPoint(max_num_keypoints=4096).eval().to(device)
matcher = LightGlue(features='superpoint').eval().to(device)

match_pair_lightglue_superpoint(
    feature_dir,
    extractor,
    matcher,
    image_paths,
    image_pairs,
    device,
    min_matches=20
)


# colmap implementation
#intrinsics = [2048, 2048, 1024, 1024] # [fx,fy, cx, cy] approximates 
pose_dict = defaultdict(dict)

for cluster_idx, cluster_imgpath in tqdm(clusters.items(), desc="Running COLMAP per cluster"):
    
    output_dir = os.path.join(colmap_output_dir, f"cluster_{cluster_idx}")
    maps = run_colmap_reconstruction(
    cluster_imgpath,
    feature_dir,
    cluster_idx,
    output_dir
    )    
    
    # Extract pose from mapped images
    for model_id, model in tqdm(maps.items(), desc = "Pose Extraction"):
        for image_id, image in model.images.items():
            pose_dict[cluster_idx][image.name] = {
                "rotation": image.cam_from_world.rotation.matrix(),
                "translation": image.cam_from_world.translation
            }
print(pose_dict)

Extracting DINO features: 100%|██████████| 9/9 [00:27<00:00,  3.02s/it]
Downloading: "https://github.com/cvg/LightGlue/releases/download/v0.1_arxiv/superpoint_v1.pth" to /root/.cache/torch/hub/checkpoints/superpoint_v1.pth
100%|██████████| 4.96M/4.96M [00:00<00:00, 164MB/s]
Downloading: "https://github.com/cvg/LightGlue/releases/download/v0.1_arxiv/superpoint_lightglue.pth" to /root/.cache/torch/hub/checkpoints/superpoint_lightglue_v0-1_arxiv.pth
100%|██████████| 45.3M/45.3M [00:00<00:00, 337MB/s]
Running COLMAP per cluster:   0%|          | 0/5 [00:00<?, ?it/s]

/kaggle/working/result/output/colmap/cluster_2/colmap.db



Add_Keypoints: 100%|██████████| 130/130 [00:02<00:00, 63.20it/s]

Importing Matches:   3%|▎         | 251/7626 [00:00<00:03, 1901.26it/s]
I20250611 15:56:20.977441 134966000297088 incremental_pipeline.cc:237] Loading database
I20250611 15:56:20.979466 134966000297088 database_cache.cc:66] Loading cameras...
I20250611 15:56:20.979550 134966000297088 database_cache.cc:76]  23 in 0.000s
I20250611 15:56:20.979588 134966000297088 database_cache.cc:84] Loading matches...
I20250611 15:56:20.980509 134966000297088 database_cache.cc:89]  251 in 0.001s
I20250611 15:56:20.980534 134966000297088 database_cache.cc:105] Loading images...
I20250611 15:56:20.982132 134966000297088 database_cache.cc:153]  23 in 0.002s (connected 23)
I20250611 15:56:20.982156 134966000297088 database_cache.cc:164] Loading pose priors...
I20250611 15:56:20.982296 134966000297088 database_cache.cc:175]  0 in 0.000s
I20250611 15:56:20.982323 134966000297088 database_cache.cc:184] Building correspondence graph...
I20250611

[DEBUG] Total valid pairs added to DB: 251


I20250611 15:56:21.176538 134966000297088 incremental_pipeline.cc:390] Registering image #20 (3)
I20250611 15:56:21.176557 134966000297088 incremental_pipeline.cc:393] => Image sees 761 / 2193 points
I20250611 15:56:21.523490 134966000297088 incremental_pipeline.cc:42] Retriangulation and Global bundle adjustment
I20250611 15:56:21.692805 134966000297088 incremental_pipeline.cc:390] Registering image #13 (4)
I20250611 15:56:21.692842 134966000297088 incremental_pipeline.cc:393] => Image sees 789 / 1889 points
I20250611 15:56:22.291907 134966000297088 incremental_pipeline.cc:42] Retriangulation and Global bundle adjustment
I20250611 15:56:22.553679 134966000297088 incremental_pipeline.cc:390] Registering image #23 (5)
I20250611 15:56:22.553708 134966000297088 incremental_pipeline.cc:393] => Image sees 820 / 2036 points
I20250611 15:56:23.150279 134966000297088 incremental_pipeline.cc:42] Retriangulation and Global bundle adjustment
I20250611 15:56:23.328865 134966000297088 incremental_p

...

/kaggle/working/result/output/colmap/cluster_0/colmap.db



Add_Keypoints: 100%|██████████| 130/130 [00:01<00:00, 103.02it/s] 

Importing Matches:   3%|▎         | 251/7626 [00:00<00:03, 2100.59it/s]
I20250611 15:59:53.626831 134966000297088 incremental_pipeline.cc:237] Loading database
I20250611 15:59:53.628455 134966000297088 database_cache.cc:66] Loading cameras...
I20250611 15:59:53.628516 134966000297088 database_cache.cc:76]  26 in 0.000s
I20250611 15:59:53.628528 134966000297088 database_cache.cc:84] Loading matches...
I20250611 15:59:53.629306 134966000297088 database_cache.cc:89]  251 in 0.001s
I20250611 15:59:53.629344 134966000297088 database_cache.cc:105] Loading images...
I20250611 15:59:53.631273 134966000297088 database_cache.cc:153]  26 in 0.002s (connected 26)
I20250611 15:59:53.631299 134966000297088 database_cache.cc:164] Loading pose priors...
I20250611 15:59:53.631480 134966000297088 database_cache.cc:175]  0 in 0.000s
I20250611 15:59:53.631494 134966000297088 database_cache.cc:184] Building correspondence graph...
I202506

[DEBUG] Total valid pairs added to DB: 251


I20250611 15:59:53.891597 134966000297088 incremental_pipeline.cc:390] Registering image #8 (3)
I20250611 15:59:53.891630 134966000297088 incremental_pipeline.cc:393] => Image sees 491 / 2695 points
I20250611 15:59:54.146093 134966000297088 incremental_pipeline.cc:42] Retriangulation and Global bundle adjustment
I20250611 15:59:54.708233 134966000297088 incremental_pipeline.cc:390] Registering image #12 (4)
I20250611 15:59:54.708264 134966000297088 incremental_pipeline.cc:393] => Image sees 273 / 1835 points
I20250611 15:59:55.261677 134966000297088 incremental_pipeline.cc:42] Retriangulation and Global bundle adjustment
I20250611 15:59:58.177707 134966000297088 incremental_pipeline.cc:390] Registering image #14 (5)
I20250611 15:59:58.177743 134966000297088 incremental_pipeline.cc:393] => Image sees 376 / 1177 points
I20250611 15:59:58.764828 134966000297088 incremental_pipeline.cc:42] Retriangulation and Global bundle adjustment
I20250611 16:00:01.689030 134966000297088 incremental_pi

defaultdict(<class 'dict'>, {2: {'imc2023_haiper/fountain_image_000.png': {'rotation': array([[ 0.2812414 ,  0.56062236, -0.77884905],
       [-0.42965002,  0.79927949,  0.42018229],
       [ 0.85808166,  0.21645986,  0.46566189]]), 'translation': array([-0.5145014 , -1.93426688,  4.85708951])}, 'imc2023_haiper/fountain_image_007.png': {'rotation': array([[-0.21223209,  0.47154846, -0.85592032],
       [-0.55052435,  0.66596393,  0.5034034 ],
       [ 0.80739116,  0.57804333,  0.11825999]]), 'translation': array([-0.48398154, -2.25682892,  3.67620904])}, 'imc2023_haiper/fountain_image_012.png': {'rotation': array([[-0.51097782,  0.40954654, -0.75576008],
       [-0.46751819,  0.60538671,  0.64415345],
       [ 0.72133793,  0.68247971, -0.11786873]]), 'translation': array([-0.25045314, -1.80196268,  3.16269412])}, 'imc2023_haiper/fountain_image_025.png': {'rotation': array([[-0.99530412, -0.05749519, -0.07787175],
       [-0.09615363,  0.49463827,  0.86376355],
       [-0.01114391,  0.8